# Chapter 5 - Django Forms

## Django Form Structure and Workflow

In [ ]:
from django import forms
from django.shortcuts import render
from django.http import HttpResponseRedirect


6.1. Django form class definition

In [ ]:
# forms.py in 'contact' app 

class ContactForm(forms.Form):
    name = forms.CharField(required=False)
    email = forms.EmailField(label='Your email')
    comment = forms.CharField(widget=forms.Textarea)


6.2. Django view method that uses a Django form

In [ ]:
# views.py in 'contact' app
from .forms import ContactForm


def contact(request):
    form = ContactForm()
    return render(request,'about/contact.html', {'form':form})


6.3. Django form instance rendered in template as HTML

In [ ]:
<tr>
    <th><label for="id_name">Name:</label></th>
    <td><input id="id_name" name="name" type="text"/></td>
</tr>
<tr>
    <th><label for="id_email">Your email:</label></th>
    <td><input id="id_email" required name="email" type="email"/></td>
</tr>
<tr>
    <th><label for="id_comment">Comment:</label></th>
    <td>
        <textarea cols="40" id="id_comment" required name="comment" rows="10"></textarea>
    </td>
</tr>


## Functional Web Form Syntax for Django Forms

6.4. Django form template declaration for functional web form

In [ ]:
<form method="POST">
    {% csrf_token %}
    <table>
        {{form.as_table}}
    </table>
    <input type="submit" value="Submit form">
</form>


## Django View Method to Process Form (POST Handling)

In [ ]:
from django.views.decorators.csrf import csrf_exempt, csrf_protect


6.5. Django view method that sends and processes Django form


In [ ]:
def contact(request):
    if request.method == 'POST':
        # POST, generate form with data from the request
        form = ContactForm(request.POST)
        # check if it's valid:
        if form.is_valid():
            # process data, insert into DB, generate email,etc
            # redirect to a new URL:
            return HttpResponseRedirect('/about/contact/thankyou')
    else:
        # GET, generate blank form
        form = ContactForm()
    return render(request,'about/contact.html', {'form':form})


6.6. Django view method decorated with @csrf_exempt() to bypass CSRF enforcement


In [ ]:
@csrf_exempt
def contact(request):
    # Any POST-processing inside view method
    # ignores if there is or isn't a CSRF token
    return

6.7. Django view method decorated with @csrf_protect() to enforce CSRF when CSRF is disabled at 
the project level

In [ ]:
@csrf_protect
def contact(request):
    # Any POST processing inside view method
    # checks for the presence of a CSRF token
    # even when CsrfViewMiddleware is removed
    return


## Django Form Processing: Initialization, Field Access, Validation, and Error Handling

In [ ]:
import re
from django import forms
from django.shortcuts import render
from django.http import HttpResponseRedirect


6.8. Django form class with backing processing view method

In [ ]:
class ContactForm(forms.Form):
    name = forms.CharField(required=False)
    email = forms.EmailField(label='Your email')
    comment = forms.CharField(widget=forms.Textarea)


def contact(request):
    if request.method == 'POST':
        form = ContactForm(request.POST)
    else:
        form = ContactForm()
    return render(request, 'about/contact.html', {'form':form})


6.9. Django form instance with initial argument declared in view method

In [ ]:
def contact(request):
    if request.method == 'POST':
        #...
        #...
        pass
    else:
        # GET,  generate blank form
        form = ContactForm(initial={'email':'johndoe@coffeehouse.com', 'name':'John Doe'})
    return render(request, 'about/contact.html', {'form':form})


6.10. Django form fields with initial argument

In [ ]:
class ContactForm(forms.Form):
    name = forms.CharField(required=False, initial='Please provide your name')
    email = forms.EmailField(label='Your email',  initial='We need your email')
    comment = forms.CharField(widget=forms.Textarea)


6.11. Django form initialized with __init__ method

In [ ]:
class ContactForm(forms.Form):
    name = forms.CharField(required=False)
    email = forms.EmailField(label='Your email')
    comment = forms.CharField(widget=forms.Textarea)

    def __init__(self, *args, **kwargs):
        initial_arguments = kwargs.get('initial', None)
        updated_initial = {}
        if initial_arguments:
            user = initial_arguments.get('user',None)
            if user:
                updated_initial['name'] = getattr(user, 'first_name', None)
                updated_initial['email'] = getattr(user, 'email', None)
        updated_initial['comment'] = 'Please provide a comment'
        kwargs.update(initial=updated_initial)
        super(ContactForm, self).__init__(*args, **kwargs)


6.12. Django form with automatic ids (default auto_id=True option) and no automatic ids auto_id=False option


In [ ]:
<!-- Option 1, default auto_id=True -->

<tr>
    <th><label for="id_name">Name:</label></th>
    <td><input id="id_name" name="name" type="text" /></td>
</tr>
<tr>
    <th><label for="id_email">Your email:</label></th>
    <td><input id="id_email" name="email" type="email" /></td>
</tr>
<tr>
    <th><label for="id_comment">Comment:</label></th>
    <td><textarea cols="40" id="id_comment" name="comment" rows="10"></textarea></td>
</tr>

<!-- Option 2 auto_id=False -->

<tr>
    <th>Name:</th><td><input name="name" type="text" /></td>
</tr>
<tr>
    <th>Your email:</th><td><input name="email" type="email" /></td>
</tr>
<tr>
    <th>Comment:</th><td><textarea cols="40" name="comment" rows="10">\r\n</textarea></td>
</tr>


6.13. Django form is_valid() method for form processing

In [ ]:
def contact(request):
    if request.method == 'POST':
        form = ContactForm(request.POST)
        if form.is_valid():
            first_name = form.cleaned_data['first_name']
            email = form.cleaned_data['email']
            return HttpResponseRedirect('/about/contact/thankyou')
        else:
            pass
    else:
        form = ContactForm()
        return render(request, 'about/contact.html', {'form':form})


6.14. Django form field validators option with custom validator method for form processing

In [ ]:
def validate_comment_word_count(value):
    count = len(value.split())
    if count < 30:
        raise forms.ValidationError(
            ('Please provide at least a 30 word message, \%(count)s words is not descriptive enough'), 
            params={'count': count},
        )


class ContactForm(forms.Form):
    name = forms.CharField(required=False)
    email = forms.EmailField(label='Your email')
    comment = forms.CharField(widget=forms.Textarea, validators=[validate_comment_word_count])


6.15. Django form field validation with clean_<field>() methods

In [ ]:
class ContactForm(forms.Form):
    name = forms.CharField(required=False)
    email = forms.EmailField(label='Your email')
    comment = forms.CharField(widget=forms.Textarea)

    def clean_name(self):
        value = self.cleaned_data['name']
        if value.isupper():
            raise forms.ValidationError(
                "Please don't use all upper case for your name, use lower case", 
                code='uppercase'
            )
        return value

    def clean_email(self):
        value = self.cleaned_data['email']
        if value.endswith('@hotmail.com'):
            raise forms.ValidationError(
                "Please don't use a hotmail email, we simply don't like it", 
                code='hotmail'
            )
        return value
    

6.16. Django form field validation with clean() method

In [ ]:
class ContactForm(forms.Form):
    name = forms.CharField(required=False)
    email = forms.EmailField(label='Your email')
    comment = forms.CharField(widget=forms.Textarea)

    def clean(self):
        super(ContactForm, self).clean()
        name = self.cleaned_data.get('name', '')
        email = self.cleaned_data.get('email', '')
        if name.lower() not in email:
            raise forms.ValidationError(
                "Please provide an email that contains your name, or viceversa"
            )


6.17. Django form field error assignment with add_error() in clean() method

In [ ]:
def clean(self):
    super(ContactForm, self).clean()
    name = self.cleaned_data.get('name', '')
    email = self.cleaned_data.get('email', '')
    if name.lower() not in email:
        message = "Please provide an email that contains your name, or viceversa"
        self.add_error('name', message)
        self.add_error('email', forms.ValidationError(message))
        self.add_error(None, message)


6.18. Django form ValidationError instance creation


In [ ]:
# Placed inside def clean_email(self):
raise forms.ValidationError(
    "Please don't use a hotmail email, we simply don't like it",
    code='hotmail'
)

# Placed inside def clean(self):
raise forms.ValidationError([
    forms.ValidationError(
        "Please provide an email that matches your name, or viceversa", 
        code='custom'
    ),
    forms.ValidationError(
        "Please provide your professional email, %(value)s doesn't look professional ", 
        code='required', 
        params={'value': self.cleaned_data.get('email')}
    ),
])


## Set Up the Layout for Django Forms in Templates

In [ ]:
from django import forms

6.19. Django form class definition

In [ ]:
class ContactForm(forms.Form):
    name = forms.CharField(required=False)
    email = forms.EmailField(label='Your email')
    comment = forms.CharField(widget=forms.Textarea)

6.20. Django form output with form.as_table

In [ ]:
<tr>
   <th><label for="id_name">Name:</label></th>
   <td><input id="id_name" name="name" type="text" /></td>
</tr>
<tr>
   <th><label for="id_email">Your email:</label></th>
   <td><input id="id_email" name="email" type="email" required/></td>
</tr>
<tr>
   <th><label for="id_comment">Comment:</label></th>
   <td><textarea cols="40" id="id_comment" name="comment" rows="10" required>\r\n</
       textarea>
   </td>
</tr>



6.21. Django form output with form.as_p


In [ ]:
<p>
    <label for="id_name">Name:</label> <input id="id_name" name="name" type="text" />
</p>
<p>
    <label for="id_email">Your email:</label> <input id="id_email" name="email" type="email" 
        required/>
</p>
<p>
    <label for="id_comment">Comment:</label> <textarea cols="40" id="id_comment" 
        name="comment" rows="10" required>\r\n</textarea>
</p>

6.22. Django form output with form.as_ul


In [ ]:
<li>
    <label for="id_name">Name:</label> <input id="id_name" name="name" type="text" />
</li>
<li>
    <label for="id_email">Your email:</label> <input id="id_email" name="email" type="email" 
        required/>
</li>
<li><label for="id_comment">Comment:</label> <textarea cols="40" id="id_comment" 
    name="comment" rows="10" required></textarea>
</li>



6.23. Django form {% for %} loop over all fields

In [ ]:
{% for field in form %}
    <div class="row">
        <div class="col-md-2">
            {{ field.label_tag }}
            {% if field.help_text %}
                <sup>{{ field.help_text }}</sup>
            {% endif %}
            {{ field.errors }}
        </div>
        <div class="col-md-10 pull-left">
            {{ field }}
        </div>
    </div>
{% endfor %}


6.24. Django form field_order option to enforce field order


In [ ]:
class ContactForm(forms.Form):
    name = forms.CharField(required=False)
    email = forms.EmailField(label='Your email')
    comment = forms.CharField(widget=forms.Textarea)
    field_order = ['email','comment','name']

6.25. Django form error_css_class and required_css_class fields to apply CSS formatting


In [ ]:
class ContactForm(forms.Form):
    name = forms.CharField(required=False)
    email = forms.EmailField(label='Your email')
    comment = forms.CharField(widget=forms.Textarea)
    error_css_class = 'error'
    required_css_class = 'bold'

6.27. Django loop over form.<field_name>.errors

In [ ]:
{% for field in form %}
    <div class="row">
        <div class="col-md-2">
            {{ field.label_tag }}
            {% if field.help_text %}
                <sup>{{ field.help_text }}</sup>
            {% endif %}
            {% for error in field.errors %}
                <div class="row">
                    <div class="alert alert-danger">{{error}}</div>
                </div>
            {% endfor %}
        </div>
        <div class="col-md-10 pull-left">
            {{ field }}
        </div>
    </div>
{% endfor %}


6.28. Django form.errors and form.non_field_errors with custom HTML output


In [ ]:
<!-- Field errors -->

{% if form.errors %}
    <div class="row">
    {% for field_with_error, error_messages in form.errors.items %}
        <div class="alert alert-danger">{{field_with_error}} {{error_messages}}</div>
    {% endfor %}
    </div>
{% endif %}

<!-- Non-field errors -->

{% if form.non_field_errors %}
    <div class="row">
    {% for error in form.non_field_errors %}
        <div class="alert alert-danger">{{error}}</div>
    {% endfor %}
    </div>
{% endif %}


## Django Custom Form Fields and Widgets

6.29. Django custom form field inherits behavior from forms.ChoiceField


In [ ]:
class GenderField(forms.ChoiceField):
    def __init__(self, *args, **kwargs):
        super(GenderField, self).__init__(*args, **kwargs)
        self.error_messages = {"required": "Please select a gender, it's required"}
        self.choices = ((None, 'Select gender'), ('M', 'Male'), ('F', 'Female'))


6.30. Django custom form widget inherits behavior from forms.widgets.Input

In [ ]:
class PlaceholderInput(forms.widgets.Input):
    template_name = 'about/placeholder.html'
    input_type = 'text'

    def get_context(self, name, value, attrs):
        context = super(PlaceholderInput, self).get_context(name, value, attrs)
        context['widget']['attrs']['maxlength'] = 50
        context['widget']['attrs']['placeholder'] = name.title()
        return context


6.31. Django custom form widget inherits behavior from forms.widgets.Input

In [ ]:
# about/placeholder.html

<input type="{{ widget.type }}" name="{{ widget.name }}"
    {% if widget.value != None %} 
        value="{{ widget.value }}"
    {% endif %}
{% include "django/forms/widgets/attrs.html" %} />

# django/forms/widgets/attrs.html

{% for name, value in widget.attrs.items %}
    {% if value is not False %} 
        {{ name }}
        {% if value is not True %}
            ="{{ value }}"
        {% endif %}
    {% endif %}
{% endfor %}
    

## Django Advanced Form Processing: Partial Forms, AJAX, and Files

In [ ]:
from coffeehouse.about.forms import ContactForm
from django.http import HttpResponse, JsonResponse
from django.conf import settings



6.32. Django form with fields marked as hidden


In [ ]:
<form method="POST">
    {% csrf_token %}
        <div class="row">
            <div class="col-md-2">
                {{ form.comment.label_tag }}
                    {% if form.comment.help_text %}
                        <sup>{{ form.comment.help_text }}</sup>
                    {% endif %}
                {% for error in form.comment.errors %}
                    <div class="row">
                        <div class="alert alert-danger">{{error}}</div>
                    </div>
                {% endfor %}
            </div>
            <div class="col-md-10 pull-left">
                {{ form.comment }}
            </div>
        </div>
        {{form.name.as_hiddden}} #### HIDDEN
        {{form.email.as_hidden}} #### HIDDEN
    <input type="submit" value="Submit form" class="btn btn-primary">
</form>


6.33. Django form subclass with removed parent fields


In [ ]:
class ContactCommentOnlyForm(ContactForm):
    def __init__(self, *args, **kwargs):
        super(ContactCommentOnlyForm, self).__init__(*args, **kwargs)
        del self.fields['name']
        del self.fields['email']


6.34. JavaScript jQuery logic to submit Django form via AJAX

In [ ]:
# NOTE: The following is only the Django (HTML) template with AJAX logic

<h4>Feedback</h4>
<div class="row">
    <div id="feedbackmessage"</div>
</div>
<form method="POST" id="feedbackform" action="{% url 'stores:feedback' %}">
    {% csrf_token %}
    <div class="row">
        <div class="col-md-12 pull-left">
            {{ form.comment }}
        </div>
    </div>
    {{form.name.as_hiddden}}
    {{form.email.as_hidden}}
    <input type="submit" value="Submit feedback" class="btn btn-primary">
</form>
<script>
    $(document).ready(function() {
        $("#feedbackform").submit(function(event) {
            event.preventDefault();
            $.ajax({data: $(this).serialize(),
                    type: $(this).attr('method'),
                    url: $(this).attr('action'),
                    success: function(response) {
                        console.log(response);
                        if(response['success']) {
                            $("#feedbackmessage").html(
                                "<div class='alert alert-success'> Succesfully sent feedback, thank you!</div>"
                            );
                            $("#feedbackform").addClass("hidden");
                        }
                        if(response['error']) {
                            $("#feedbackmessage").html(
                                "<div class='alert alert-danger'>" + response['error']['comment'] + "</div>");
                        }
                    },
                    error: function (request, status, error) {
                        console.log(request.responseText);
                    }
            });
        });
    })
</script>

6.35. Django view method to process Django form via AJAX


In [ ]:
# urls.py (Main)

urlpatterns = [
    url(r'^stores/', include('coffeehouse.stores.urls', namespace="stores"))
]

# urls.py (App stores)

urlpatterns = [
    url(r'^$', views.index, name="index"), 
    url(r'^feedback/$', views.feedback, name="feedback"), 
]

# views.py (App stores)

def feedback(request):
    if request.POST:
        form = ContactForm(request.POST)
        if form.is_valid():
            return JsonResponse({'success':True})
        else:
            return JsonResponse({'error':form.errors})
    return HttpResponse("Hello from feedback!")

6.36. Django form with file fields, corresponding view method, and template layout


In [ ]:
# forms.py

class SharingForm(forms.Form):
    # NOTE: forms.PhotoField requires Python PIL & other operating system libraries, 
    # so generic FileField is used instead
    video = forms.FileField()
    photo = forms.FileField(widget=forms.ClearableFileInput(attrs={'multiple': True}))

# views.py

def index(request):
    if request.method == 'POST':
        form = SharingForm(request.POST, request.FILES)
        if form.is_valid():
            return HttpResponseRedirect('/about/contact/thankyou')
    else:
        form = SharingForm()
        return render(request, 'social/index.html', {'form':form})


In [ ]:
# social/index.html

<form method="post" enctype="multipart/form-data">
    {% csrf_token %}
    <ul>
        {{form.as_ul}}
    </ul>
    <input type="submit" value="Submit photo" class="btn btn-primary">
</form>


6.37. Django form file processing with save procedure to MEDIA_ROOT

In [ ]:
# views.py

def save_uploaded_file_to_media_root(f):
    with open('%s%s' % (settings.MEDIA_ROOT, f.name), 'wb+') as destination:
        for chunk in f.chunks():
            destination.write(chunk)

def index(request):
    if request.method == 'POST':
        form = SharingForm(request.POST, request.FILES)
        if form.is_valid():
            for field in request.FILES.keys():
                for formfile in request.FILES.getlist(field):
                    save_uploaded_file_to_media_root(formfile)
        return HttpResponseRedirect('/about/contact/thankyou')
    else:
        form = SharingForm()
        return render(request, 'social/index.html', {'form':form})


## Django Formsets

In [ ]:
from django.forms import formset_factory
from django.forms import BaseFormSet


6.38. Django formset factory initialization and template layout


In [ ]:
# forms.py

DRINKS = ((None, 'Please select a drink type'), (1, 'Mocha'), (2, 'Espresso'), (3, 'Latte'))
SIZES = ((None, 'Please select a drink size'), ('s', 'Small'), ('m', 'Medium'), ('l', 'Large'))

class DrinkForm(forms.Form):
    name = forms.ChoiceField(choices=DRINKS, initial=0)
    size = forms.ChoiceField(choices=SIZES, initial=0)
    amount = forms.ChoiceField(
        choices=[(None, 'Amount of drinks')] + [(i, i) for i in range(1, 10)]
    )

# views.py

def index(request):
    DrinkFormSet = formset_factory(DrinkForm, extra=2, max_num=20)
    if request.method == 'POST':
        # TODO
        pass
    else:
        formset = DrinkFormSet(initial=[{'name': 1, 'size': 'm', 'amount':1}])
        return render(request, 'online/index.html', {'formset':formset})


In [ ]:
# online/index.html

<form method="post">
    {% csrf_token %}
    {{ formset.management_form }}
    <table>
        {% for form in formset %}
            <tr><td><ul class="list-inline">{{ form.as_ul }}</ul></td></tr>
        {% endfor %}
    </table>
    <input type="submit" value="Submit order" class="btn btn-primary">
</form>


6.39. Django formset management form contents and fields


In [ ]:
<input type="hidden" name="form-TOTAL_FORMS" value="3" id="id_form-TOTAL_FORMS" />
<input type="hidden" name="form-INITIAL_FORMS" value="1" id="id_form-INITIAL_FORMS" />
<input type="hidden" name="form-MIN_NUM_FORMS" value="0" id="id_form-MIN_NUM_FORMS" />
<input type="hidden" name="form-MAX_NUM_FORMS" value="20" id="id_form-MAX_NUM_FORMS" />

6.40. Django formset designed to add extra forms by user

In [ ]:
#views.py

def index(request):
    extra_forms = 2
    DrinkFormSet = formset_factory(DrinkForm, extra=extra_forms, max_num=20)
    if request.method == 'POST':
        if 'additems' in request.POST and request.POST['additems'] == 'true':
            formset_dictionary_copy = request.POST.copy()
            formset_dictionary_copy['form-TOTAL_FORMS'] = (
                int(formset_dictionary_copy['form-TOTAL_FORMS']) + 
                extra_forms
            )
            formset = DrinkFormSet(formset_dictionary_copy)
        else:
            formset = DrinkFormSet(request.POST)
            if formset.is_valid():
                return HttpResponseRedirect('/about/contact/thankyou')
    else:
        formset = DrinkFormSet(initial=[{'name': 1, 'size': 'm', 'amount':1}])
    return render(request, 'online/index.html', {'formset':formset})


In [ ]:
# online/index.html

<form method="post">
    {% csrf_token %}
    {{ formset.management_form }}
    <table>
        {% for form in formset %}
            <tr><td>{{ form }}</td></tr>
        {% endfor %}
    </table>
    <input type="hidden" value="false" name="additems" id="additems">
    <button class="btn btn-primary" id="additemsbutton">Add items to order</button>
    <input type="submit" value="Submit order" class="btn btn-primary">
</form>
<script>
    $(document).ready(function() {
        $("#additemsbutton").on(
            'click', 
            function(event) {
                $("#additems").val("true");
        });
    });
</script>

6.41. Django custom formset with custom validation

In [ ]:
class BaseDrinkFormSet(BaseFormSet):
    def clean(self):
        if any(self.errors): 
            return # If there are any error, no point in validating further
        name_size_tuples = []
        for form in self.forms:
            name_size = (form.cleaned_data['name'], form.cleaned_data['size'])
            if name_size in name_size_tuples:
                raise forms.ValidationError(
                    "Ups! You have multiple %s %s items in your order, keep one and increase the amount" % 
                    (dict(SIZES)[name_size[1]], dict(DRINKS)[int(name_size[0])])
                )
            name_size_tuples.append(name_size)


6.42. Django custom formset to display non_form_errors

In [ ]:
<form method="post">
    {% csrf_token %}
    {{ formset.management_form }}
    {% if formset.non_form_errors %}
        <div class="alert alertdanger">{{formset.non_form_errors}}</div>
    {% endif %}
    {{ formset.management_form }}
    <table>
        {% for form in formset %}
            <tr><td><ul class="list-inline">{{ form.as_ul }}</ul></td></tr>
        {% endfor %}
    </table>
</form>
